In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
import pandas as pd

url = "https://m.imdb.com/title/tt6468322/reviews"
driver.get(url)

In [ ]:
button = driver.find_element_by_class_name("ipl-load-more")

while True:
  try:
    list_of_spoilers = driver.find_elements_by_xpath("//div[@class='ipl-expander spoiler-warning']")
    for i in list_of_spoilers:
      i.click()
    button.click()
    button = driver.find_element_by_class_name("ipl-load-more")
  except ElementNotInteractableException:
    break

print("all reviews loaded")


all reviews loaded


In [ ]:
df = pd.DataFrame(columns=["rating","title","text","sentiment"])
list_of_reviews = driver.find_elements_by_class_name("ipl-content-list__item")

for review in list_of_reviews:
  try:
    ratingele = review.find_element_by_class_name("rating-other-user-rating")
    rating = int(ratingele.find_element_by_tag_name("span").text)
  except NoSuchElementException:
    continue
  
  if 8>rating>4:
    continue
  
  title = review.find_element_by_class_name("title").text
  text = review.find_element_by_class_name("text").text

  if len(text.split())>100:
    continue
  
  df = df.append({
      "rating":rating,
      "title":title,
      "text":text,
      "sentiment":0 if rating<=4 else 1
  },ignore_index=True)

df.to_csv("data.csv",index=False,sep="\t")